In [1]:
import numpy as np
from profiles import *
from dispersion import SphericalJeansDispersion
from likelihood import GaussianLikelihood
from fitter import MinuitFitter
from utils import *

In [2]:
homedir = '/home/andrea/Desktop/work/DWARF/Jvalue/project1/test/Isotrop_Core_nonPlum'
MockSize = 100
dataSize = 100
dset = 1
# enter model choice - cf. casedir (options 1,2,3,4)
mod = 1
theta = 0.5
D = 100

In [3]:
data = load_gaia(homedir, MockSize, dataSize, dset, mod, D, True)

Using  /home/andrea/Desktop/work/DWARF/Jvalue/project1/test/Isotrop_Core_nonPlum/data/gs100_bs050_rcrs100_rarcinf_core_0400mpc3_df_100_1.dat


In [4]:
kinematic = data[0:3]
rh = data[-3]
r0_true = data[-2]
rho0_true = data[-1]

In [5]:
dm = build_profile('Zhao', a=1., b=3., c=0.) # Cored DM profile
st = build_profile('plummer', **{'c':1, 'rh':rh}) # non-Plummer Stellar profile
kr = build_kernel('iso') # isotropic kernel

In [6]:
dwarf_props = {'D':D, 'theta':theta, 'rt':np.inf, 'errs':False}

In [7]:
Sigma = SphericalJeansDispersion(dm, st, kr, dwarf_props)

In [8]:
LL = GaussianLikelihood(kinematic, Sigma)

In [9]:
LL.set_free('dm_r0')

In [10]:
LL.sigma.setparams('J',18)

In [11]:
global global_loglike
global_loglike = LL
M = MinuitFitter(LL)

In [12]:
M.set_value('J',19)
M.set_value('dm_r0',rh/2.)
M.set_error('J',0.01)
M.set_bound('J',(15,22))
M.set_error('dm_r0',0.01)
M.set_bound('dm_r0',(kinematic[0].min(),kinematic[0].max()))
M.settings['print_level'] = 1

In [13]:
M.settings

{'J': 19,
 'dm_r0': 0.5,
 'error_J': 0.01,
 'error_dm_r0': 0.01,
 'errordef': 0.5,
 'limit_J': (15, 22),
 'limit_dm_r0': (0.075304685777181218, 8.2368209066605313),
 'pedantic': False,
 'print_level': 1}

In [14]:
M.set_minuit(**{'tol':1e2,'strategy':2})

In [15]:
M.minos_profile('J')

Function not at minimum. Running migrad first.


FCN = 352.587189415 
 TOTAL NCALL = 89 
 NCALLS = 89 
 
 
 EDM = 2.2583920826e-05 
 GOAL EDM = 0.005 
 
 UP = 0.5 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 True 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 False 
 
 False

+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,J,1.923088e+01,1.719311e-01,0.000000e+00,0.000000e+00,15.0,22.0,
2,dm_r0,6.250644e-01,2.342044e-01,0.000000e+00,0.000000e+00,0.0753046857772,8.23682090666,


+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,J,1.923088e+01,1.435789e-01,0.000000e+00,0.000000e+00,15.0,22.0,
2,dm_r0,6.250644e-01,8.519177e-04,0.000000e+00,0.000000e+00,0.0753046857772,8.23682090666,


+,J,dm_r0
J,1.00,0.51
dm_r0,0.51,1.00


(array('d', [18.94372186621972, 18.963525848739575, 18.983329831259432, 19.003133813779286, 19.022937796299143, 19.042741778818996, 19.06254576133885, 19.082349743858707, 19.10215372637856, 19.121957708898417, 19.14176169141827, 19.161565673938128, 19.18136965645798, 19.201173638977835, 19.220977621497692, 19.240781604017545, 19.260585586537402, 19.280389569057256, 19.30019355157711, 19.319997534096967, 19.33980151661682, 19.359605499136677, 19.37940948165653, 19.399213464176388, 19.41901744669624, 19.438821429216095, 19.45862541173595, 19.478429394255805, 19.498233376775662, 19.518037359295516]),
 array('d', [354.10071321996054, 353.87851195996285, 353.67669201396274, 353.4945234099317, 353.3312860990725, 353.1862697883324, 353.0587735959677, 352.94810640908827, 352.8535860901462, 352.7745395827632, 352.71030291927985, 352.6602212285797, 352.62364874174847, 352.5999490384717, 352.58849657911617, 352.5886708642574, 352.5998672976568, 352.62149033980216, 352.6529546091185, 352.693687284